In [1]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import torch

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
model = GPT2LMHeadModel.from_pretrained("output/version_4/checkpoint-54000").to(device)
tokenizer = PreTrainedTokenizerFast.from_pretrained("gunwoo723/kogpt2-generate-poem",
  bos_token='<s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

In [17]:
sentence = '<s>떠나가는 새들을 보며 혼자남겨진 나\n'
input_ids = tokenizer.encode(sentence, add_special_tokens=True, return_tensors='pt').to(device)

with torch.no_grad():
    output = model.generate(
        input_ids=input_ids,
        temperature=0.5, # 생성 다양성 조절
        max_length=256, # 생성되는 문장의 최대 길이
        repetition_penalty=1.2, # 반복을 줄이는 패널티
        do_sample=True, # 샘플링 기반 생성 활성화
        early_stopping=True, # EOS token을 만나면 조기 종료
        eos_token_id=tokenizer.eos_token_id
    )

generated_sequence = output[0].tolist()
text = tokenizer.decode(generated_sequence, skip_special_tokens=True)

In [18]:
print(text)

떠나가는 새들을 보며 혼자남겨진 나
밤하늘 별빛에 반짝이는 별을 보노라면
아직도 꿈결같은 그대의 모습이 떠오른다

어둠속에 잠긴 밤하늘의 별이 
내안에 가득차면 내안도 반짝반짝 빛나고
별빛이 되어 그대에게 향하면
나는 차라리 홀로 남은 것이 서럽습니다
